In [53]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import subprocess
import requests
import pandas as pd

BASE_URL = "https://liquipedia.net"
URL = "https://liquipedia.net/starcraft2/S-Tier_Tournaments"

HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

In [ ]:
resp = requests.get(URL, headers = HEADERS)
soup = BeautifulSoup(resp.text, "html.parser")

In [45]:
tournament_links = []

for cell in soup.select("div.gridCell.Tournament.Header"):
    a = cell.find("a", href=True)
    if a:
        tournament_links.append(
            urljoin(BASE_URL, a["href"])
        )
    # end if
# end for

for link in tournament_links[3:10]:
    print(link)
# end for

https://liquipedia.net/starcraft2/Esports_World_Cup/2025
https://liquipedia.net/starcraft2/Global_StarCraft_II_League
https://liquipedia.net/starcraft2/2025_DreamHack/Dallas
https://liquipedia.net/starcraft2/Global_StarCraft_II_League
https://liquipedia.net/starcraft2/Gold_Series_Team_Championship
https://liquipedia.net/starcraft2/Esports_World_Cup/2024
https://liquipedia.net/starcraft2/Global_StarCraft_II_League


In [47]:
link = tournament_links[3]
resp = requests.get(link, headers = HEADERS) 
soup = BeautifulSoup(resp.text, "html.parser")

In [ ]:
tier = None

tier_block = soup.select_one("div.valvepremier-highlighted")
if tier_block:
    a = tier_block.find("a")
    if a:
        tier = a.get_text(strip=True)
    # end if
# end if
        
print(tier)

S-Tier


In [49]:
aligulac_url = None

a = soup.select_one("a:has(i.lp-aligulac)")
if a:
    aligulac_url = a["href"]
# end if

print(aligulac_url)

http://aligulac.com/results/events/152545


In [51]:
resp = requests.get(aligulac_url, headers = HEADERS) 
soup = BeautifulSoup(resp.text, "html.parser")

In [ ]:
matches = []

for tbody in soup.select("tbody.lm"):
    for row in tbody.select("tr"):
        date_cell = row.select_one("td.lm_date")
        if not date_cell:
            continue
        date = date_cell.get_text(strip=True)

        score_match = row.select_one("td.lm_score").get_text(strip=True)
        p_score, o_score = score_match.replace("–", "-").split("-")
        p_score = int(p_score)
        o_score = int(o_score)

        pla = row.select_one("td.lm_pla")
        pla_a = pla.find("a")
        player = pla_a.get_text(strip=True)
        player_id = int(pla_a["href"].split("/")[2].split("-")[0])
        player_race = pla.find("img", alt=lambda x: x in {"P", "T", "Z"}).get("alt")

        plb = row.select_one("td.lm_plb")
        plb_a = plb.find("a")
        opponent = plb_a.get_text(strip=True)
        opponent_id = int(plb_a["href"].split("/")[2].split("-")[0])
        opponent_race = plb.find("img", alt=lambda x: x in {"P", "T", "Z"}).get("alt")

        if "winner" in pla.get("class", []):
            winner_score, loser_score = p_score, o_score
        else:
            winner_score, loser_score = o_score, p_score
        # end if

        matches.append({
            "date": date,
            "player": player,
            "player_race": player_race,
            "player_score": winner_score if "winner" in pla.get("class", []) else loser_score,
            "player_id": player_id,
            "opponent": opponent,
            "opponent_race": opponent_race,
            "opponent_score": loser_score if "winner" in pla.get("class", []) else winner_score,
            "opponent_id": opponent_id,
            "score_match": score_match
        })
    # end for
# end for

df = pd.DataFrame(matches)

,date,player,player_race,player_score,player_id,opponent,opponent_race,opponent_score,opponent_id,score_match
0,07/25/2025,Serral,Z,5,485,Classic,P,2,186,5–2
1,07/25/2025,Cure,T,4,1665,Solar,Z,0,1793,4–0
2,07/25/2025,Solar,Z,1,1793,Classic,P,4,186,1–4
3,07/25/2025,Serral,Z,4,485,Cure,T,0,1665,4–0
4,07/24/2025,Serral,Z,3,485,Reynor,Z,2,5414,3–2
5,07/24/2025,Clem,T,0,5878,Classic,P,3,186,0–3
6,07/24/2025,herO,P,0,233,Solar,Z,3,1793,0–3
7,07/24/2025,Maru,T,1,49,Cure,T,3,1665,1–3
8,07/24/2025,Cure,T,3,1665,Rogue,Z,0,1662,3–0
9,07/24/2025,Classic,P,3,186,ShoWTimE,P,0,2170,3–0
